In [1]:
import numpy as np

#PYTHON MODULES
from spectral_sirens.detectors import sensitivity_curves as sc
from spectral_sirens.detectors import horizon

We load the sensitivity curves

In [2]:
Sn_O3, fmin_O3, fmax_O3 = sc.detector_psd('O3')
Sn_O4, fmin_O4, fmax_O4 = sc.detector_psd('O4')
Sn_Aplus, fmin_Aplus, fmax_Aplus = sc.detector_psd('A+')
Sn_Asharp, fmin_Asharp, fmax_Asharp = sc.detector_psd('A#')
Sn_CE, fmin_CE, fmax_CE = sc.detector_psd('CE-40')
based = 'ground'
snr_th = 8.

We compute the horizon distance

In [3]:
nfs = 20 #100 gives detailed results
fs_O3=np.logspace(np.log10(fmin_O3),np.log10(fmax_O3),nfs)
fs_O4=np.logspace(np.log10(fmin_O4),np.log10(fmax_O4),nfs)
fs_Aplus=np.logspace(np.log10(fmin_Aplus),np.log10(fmax_Aplus),nfs)
fs_Asharp=np.logspace(np.log10(fmin_Asharp),np.log10(fmax_Asharp),nfs)
fs_CE=np.logspace(np.log10(fmin_CE),np.log10(fmax_CE),nfs)

nms = 20 #500 gives detailed results
masses = np.logspace(np.log10(0.5),5,nms)
masses_ext = np.logspace(np.log10(0.5),7.,nms)
Tobs = 1.0

zhor_O3 = horizon.zhor_max(masses,masses,fs_O3,Tobs,Sn_O3,fmin_O3,fmax_O3,based,snr_th)
zhor_O4 = horizon.zhor_max(masses,masses,fs_O4,Tobs,Sn_O4,fmin_O4,fmax_O4,based,snr_th)
zhor_Aplus = horizon.zhor_max(masses,masses,fs_Aplus,Tobs,Sn_Aplus,fmin_Aplus,fmax_Aplus,based,snr_th)
zhor_Asharp = horizon.zhor_max(masses,masses,fs_Asharp,Tobs,Sn_Asharp,fmin_Asharp,fmax_Asharp,based,snr_th)
zhor_CE = horizon.zhor_max(masses,masses,fs_CE,Tobs,Sn_CE,fmin_CE,fmax_CE,based,snr_th)

For later use, we can save the horizon distance

In [ ]:
np.savetxt('gw_detector_horizon/zhor_max_o3.txt',np.column_stack((2.*masses/(1.+zhor_O3),zhor_O3)),delimiter = ' ')
np.savetxt('gw_detector_horizon/zhor_max_o4.txt',np.column_stack((2.*masses/(1.+zhor_O4),zhor_O4)),delimiter = ' ')
np.savetxt('gw_detector_horizon/zhor_max_aplus.txt',np.column_stack((2.*masses/(1.+zhor_Aplus),zhor_Aplus)),delimiter = ' ')
np.savetxt('gw_detector_horizon/zhor_max_asharp.txt',np.column_stack((2.*masses/(1.+zhor_Asharp),zhor_Asharp)),delimiter = ' ')
np.savetxt('gw_detector_horizon/zhor_max_ce.txt',np.column_stack((2.*masses/(1.+zhor_CE),zhor_CE)),delimiter = ' ')